In [ ]:
# --- 0) Install (run once per Colab session) ---
!pip -q install -U yt-dlp transformers accelerate
!apt-get -qq update
!apt-get -qq install -y ffmpeg

import os, subprocess, shlex, json, torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 66.3 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
# --- 1) Load Whisper Large-v3 (run once per session) ---
MODEL_ID = "openai/whisper-large-v3-turbo"
OUTDIR = "/content/audios"
os.makedirs(OUTDIR, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype  = torch.float16 if device == "cuda" else torch.float32

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    MODEL_ID, torch_dtype=dtype, low_cpu_mem_usage=True
).to(device)
processor = AutoProcessor.from_pretrained(MODEL_ID)

asr = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    device=0 if device == "cuda" else -1,
    torch_dtype=dtype,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/587 [00:00<?, ?it/s]

...

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


In [ ]:
# --- 2) Robust downloader: returns the exact WAV filepath (no glob guessing) ---
def download_youtube_audio_wav(url: str, outdir: str = OUTDIR) -> str:
    # Template so we know name; we also ask yt-dlp to PRINT the final filepath.
    outtmpl = os.path.join(outdir, "%(id)s.%(ext)s")

    # These flags often fix Colab failures (exit status 1) due to YouTube client checks / network.
    cmd = [
        "yt-dlp",
        "--force-ipv4",
        "--extractor-args", "youtube:player_client=android",
        "--no-playlist",
        "-x", "--audio-format", "wav", "--audio-quality", "0",
        "-o", outtmpl,
        "--print", "after_move:filepath",
        url
    ]

    p = subprocess.run(cmd, text=True, capture_output=True)
    if p.returncode != 0:
        # Show the real reason (most useful part is stderr)
        raise RuntimeError(
            "yt-dlp failed.\n\n"
            f"COMMAND:\n{' '.join(shlex.quote(c) for c in cmd)}\n\n"
            f"STDERR (last 2000 chars):\n{p.stderr[-2000:]}\n"
        )

    # yt-dlp prints the final path on the last non-empty line
    lines = [ln.strip() for ln in p.stdout.splitlines() if ln.strip()]
    wav_path = lines[-1]
    if not os.path.exists(wav_path):
        raise RuntimeError(f"Download reported '{wav_path}' but file not found.")
    return wav_path

In [ ]:
# --- 3) Transcribe + save TXT + JSON next to the audio ---
def transcribe_and_save(audio_path: str, language: str = "ur") -> dict:
    result = asr(
        audio_path,
        chunk_length_s=30,
        batch_size=16,
        return_timestamps=True,
        generate_kwargs={"task": "transcribe", "language": language} if language else {"task": "transcribe"},
    )

    base = os.path.splitext(audio_path)[0]
    txt_path  = base + ".txt"
    json_path = base + ".json"

    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(result.get("text", ""))

    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

    return {"audio": audio_path, "txt": txt_path, "json": json_path, "text": result.get("text", "")}

In [ ]:
# --- 4) One function: URL -> WAV -> transcript files ---
def transcribe_youtube_url(url: str, language: str = "ur"):
    audio_path = download_youtube_audio_wav(url)
    outputs = transcribe_and_save(audio_path, language=language)
    print("✅ Saved audio: ", outputs["audio"])
    print("✅ Saved text : ", outputs["txt"])
    print("✅ Saved json : ", outputs["json"])
    print("\n--- TRANSCRIPT ---\n", outputs["text"])
    return outputs

# =========================
# ONLY change this URL ↓↓↓
# =========================
URL = "https://www.youtube.com/shorts/xrPcEuLyYwc"   # or shorts link
#URL = "https://www.youtube.com/shorts/1ewNH2jBVns"
transcribe_youtube_url(URL, language="ur")

Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


✅ Saved audio:  /content/audios/xrPcEuLyYwc.wav
✅ Saved text :  /content/audios/xrPcEuLyYwc.txt
✅ Saved json :  /content/audios/xrPcEuLyYwc.json

--- TRANSCRIPT ---
  کہ رمضان تو بس دو تین دن نہیں رہ گئے ہیں رمضان میں تو اس کی آمد سے پہلے ہی پیٹرولی مصنوعات کی قیمتوں میں اضافہ ہو گیا ہے تو یہ بھی کوئی اچھی خبر نہیں ہے لیکن بتانا ضروری ہے آج سے پیٹرل جو ہے وہ پانچ روپے اور ڈیزل سات روپے بتیس پیسے فی لیٹر مہنگا ملے گا پیٹرل کی قیمت میں پانچ روپے کا اضافہ ہو گیا ہے پیٹرل کی قیمت میں پانچ روپے کا اضافہ ہو گیا ہے پیٹرل کی قیمت میں پانچ روپے کا اضافہ ہو گیا ہے پیٹرل کی قیمت میں پانچ روپے کا اضافہ ہو گیا ہے پیٹرل کی قیمت میں پانچ روپے کا اضافہ ہو گیا ہے پیٹرل کی قیمت میں پانچ روپے کا اضافہ ہو گیا ہے پیٹرل کی قیمت میں پانچ روپے کا اضافہ ہو گیا ہے ہمارے چہرے کے تاثرات خبروں کے ساتھ جان ہی رہے ہیں لیکن بس اب زندگی میں ایسے ہی نبردازمہ ہونا پڑے گا انعالات سے پیٹویل کی نئی قیمت 258 روپے سترہ پیسے فیلیٹر بکرر ہوئی ہے ہائی سپیڈ ڈیزل کی نئی قیمت 275 روپے ستر پیسے فیلیٹر ہو گئی ہے پیٹرولیم ڈیویشن نے ق

{'audio': '/content/audios/xrPcEuLyYwc.wav',
 'txt': '/content/audios/xrPcEuLyYwc.txt',
 'json': '/content/audios/xrPcEuLyYwc.json',
 'text': ' کہ رمضان تو بس دو تین دن نہیں رہ گئے ہیں رمضان میں تو اس کی آمد سے پہلے ہی پیٹرولی مصنوعات کی قیمتوں میں اضافہ ہو گیا ہے تو یہ بھی کوئی اچھی خبر نہیں ہے لیکن بتانا ضروری ہے آج سے پیٹرل جو ہے وہ پانچ روپے اور ڈیزل سات روپے بتیس پیسے فی لیٹر مہنگا ملے گا پیٹرل کی قیمت میں پانچ روپے کا اضافہ ہو گیا ہے پیٹرل کی قیمت میں پانچ روپے کا اضافہ ہو گیا ہے پیٹرل کی قیمت میں پانچ روپے کا اضافہ ہو گیا ہے پیٹرل کی قیمت میں پانچ روپے کا اضافہ ہو گیا ہے پیٹرل کی قیمت میں پانچ روپے کا اضافہ ہو گیا ہے پیٹرل کی قیمت میں پانچ روپے کا اضافہ ہو گیا ہے پیٹرل کی قیمت میں پانچ روپے کا اضافہ ہو گیا ہے ہمارے چہرے کے تاثرات خبروں کے ساتھ جان ہی رہے ہیں لیکن بس اب زندگی میں ایسے ہی نبردازمہ ہونا پڑے گا انعالات سے پیٹویل کی نئی قیمت 258 روپے سترہ پیسے فیلیٹر بکرر ہوئی ہے ہائی سپیڈ ڈیزل کی نئی قیمت 275 روپے ستر پیسے فیلیٹر ہو گئی ہے پیٹرولیم ڈیویشن نے قیمتوں میں اضافہ کا نوٹف